In [1]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16

In [2]:
image_width, image_height = 64, 64
num_classes = 164
images = []
labels = []

In [3]:
def preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((image_width, image_height))
    img = np.array(img)
    img = img / 255.0  
    return img

In [4]:
dataset_path = r'D:\Project3\tai\EarVN1.0 dataset\Images'

In [5]:
class_label_mapping = {}
class_count = 0

In [6]:
for class_name in os.listdir(dataset_path):
    class_folder = os.path.join(dataset_path, class_name)
    if not os.path.isdir(class_folder):
        continue
    class_label_mapping[class_name] = class_count
    class_count += 1

    for image_name in os.listdir(class_folder):
        image_path = os.path.join(class_folder, image_name)
        img = preprocess_image(image_path)
        images.append(img)
        labels.append(class_label_mapping[class_name])

In [7]:
class_label_mapping

{'001.ALI_HD': 0,
 '002.LeDuong_BL': 1,
 '003.BD_Tran': 2,
 '004.Binz': 3,
 '005.Bui_AT': 4,
 '006.Chau_GK': 5,
 '007.Chau_KP': 6,
 '008.Chi_D': 7,
 '009.Chi_Th': 8,
 '010.Chu_B': 9,
 '011.Cong_To': 10,
 '012.Dai_Nhan': 11,
 '013.Dam_Vinh_H': 12,
 '014.Dan_Ng': 13,
 '015.Dan_Trg': 14,
 '016.Dang_L': 15,
 '017.Dao_Ba_L': 16,
 '018.De_C': 17,
 '019.Don_Ng': 18,
 '020.Duong_D': 19,
 '021.Dustin_Phuc_Ng': 20,
 '022.Duy_Kh': 21,
 '023.Erik': 22,
 '024.Gin_Tuan_K': 23,
 '025.Ha_Anh_T': 24,
 '026.Hac_Hoa_K': 25,
 '027.Hamtet_Tr': 26,
 '028.Ho_Quang_H': 27,
 '029.Ho_Trung_D': 28,
 '030.Hoai_L': 29,
 '031.Hoang_Rap': 30,
 '032.Huy_Tr': 31,
 '033.Huynh_L': 32,
 '034.Huynh_Ph': 33,
 '035.Isaac': 34,
 '036.Jun_Ph': 35,
 '037.Justatee': 36,
 '038.Kenvin_Kh': 37,
 '039.KhacVi': 38,
 '040.Khuong_Ng': 39,
 '041.Kieu_Minh_T': 40,
 '042.Kim_L': 41,
 '043.L_HA': 42,
 '044.Lam_Canh_T': 43,
 '045.Lam_Hu': 44,
 '046.Lam_Tr': 45,
 '047.Lam_Vinh_H': 46,
 '048.Lee_Jong_S': 47,
 '049.Lee_Min_H': 48,
 '050.Long_

In [8]:
images = np.array(images)
labels = np.array(labels)

In [9]:
len(images)

28412

In [10]:
labels = to_categorical(labels, num_classes)

In [11]:
labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [12]:
indices = np.arange(images.shape[0])
np.random.shuffle(indices)
images = images[indices]
labels = labels[indices]

In [13]:
train_images, temp_images, train_labels, temp_labels = train_test_split(images, labels, test_size=0.4, random_state=42)
val_images, test_images, val_labels, test_labels = train_test_split(temp_images, temp_labels, test_size=0.5, random_state=42)

In [14]:
train_datagen1 = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

In [15]:
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_width, image_height, 3))
for layer in vgg_model.layers:
    layer.trainable = False

In [16]:
model = Sequential()

In [17]:
model = Sequential()
model.add(vgg_model)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 2, 2, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 164)               42148     
                                                                 
Total params: 15,937,252
Trainable params: 1,222,564
Non-trainable params: 14,714,688
_________________________________________________________________


In [19]:
model.compile(optimizer= Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
train_datagen2 = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator()

In [21]:
batch_size = 32
epochs = 100

In [22]:
train_generator1 = train_datagen1.flow(train_images, train_labels, batch_size=batch_size)
val_generator = val_datagen.flow(val_images, val_labels, batch_size=batch_size)

In [23]:
model.fit(train_generator1, epochs=epochs, validation_data= val_generator)

Epoch 1/100
533/533 [==============================] - 23s 31ms/step - loss: 4.9703 - accuracy: 0.0201 - val_loss: 4.7068 - val_accuracy: 0.0345
Epoch 2/100
533/533 [==============================] - 16s 30ms/step - loss: 4.6381 - accuracy: 0.0456 - val_loss: 4.4180 - val_accuracy: 0.0651
Epoch 3/100
533/533 [==============================] - 15s 28ms/step - loss: 4.4766 - accuracy: 0.0639 - val_loss: 4.2877 - val_accuracy: 0.0913
Epoch 4/100
533/533 [==============================] - 15s 29ms/step - loss: 4.3817 - accuracy: 0.0736 - val_loss: 4.1938 - val_accuracy: 0.0959
Epoch 5/100
533/533 [==============================] - 15s 28ms/step - loss: 4.3007 - accuracy: 0.0841 - val_loss: 4.1193 - val_accuracy: 0.1133
Epoch 6/100
533/533 [==============================] - 15s 27ms/step - loss: 4.2313 - accuracy: 0.0939 - val_loss: 4.0858 - val_accuracy: 0.1170
Epoch 7/100
533/533 [==============================] - 15s 28ms/step - loss: 4.1886 - accuracy: 0.1010 - val_loss: 4.0337 - val_ac

In [24]:
train_generator2 = train_datagen2.flow(train_images, train_labels, batch_size=batch_size)

In [25]:
model.fit(train_generator2, epochs=epochs, validation_data=val_generator)

Epoch 1/100
533/533 [==============================] - 14s 27ms/step - loss: 3.5918 - accuracy: 0.1933 - val_loss: 3.8810 - val_accuracy: 0.1844
Epoch 2/100
533/533 [==============================] - 15s 28ms/step - loss: 3.5547 - accuracy: 0.1935 - val_loss: 3.8666 - val_accuracy: 0.1920
Epoch 3/100
533/533 [==============================] - 15s 28ms/step - loss: 3.5673 - accuracy: 0.1991 - val_loss: 3.8170 - val_accuracy: 0.1899
Epoch 4/100
533/533 [==============================] - 14s 27ms/step - loss: 3.5630 - accuracy: 0.1937 - val_loss: 3.8413 - val_accuracy: 0.1924
Epoch 5/100
533/533 [==============================] - 15s 27ms/step - loss: 3.5730 - accuracy: 0.1907 - val_loss: 3.8382 - val_accuracy: 0.1920
Epoch 6/100
533/533 [==============================] - 14s 27ms/step - loss: 3.5520 - accuracy: 0.1968 - val_loss: 3.8789 - val_accuracy: 0.1906
Epoch 7/100
533/533 [==============================] - 14s 27ms/step - loss: 3.5602 - accuracy: 0.1909 - val_loss: 3.8321 - val_ac